## Implementación en Python del proceso de mejora de contraste:


**Explicación del código:**

* La función `enhance_contrast` toma una imagen EL como entrada y devuelve una imagen con el contraste mejorado.
* La función primero difumina la imagen con un filtro gaussiano para eliminar el ruido.
* Luego, se aplica un cierre morfológico para eliminar pequeñas imperfecciones y mejorar la uniformidad del fondo.
* La imagen original se divide por la imagen de fondo, lo que atenúa el ruido y resalta las regiones de alto contraste.
* Finalmente, se aplica la ecualización del histograma para aumentar el contraste general de la imagen.


In [1]:
# Importación de librerías

import cv2
import numpy as np

In [2]:
# Función para mejorar el contraste de una imagen EL
def enhance_contrast(image):
  """
  Mejora el contraste de una imagen EL.

  Parámetros:
    image: La imagen EL de entrada.

  Devuelve:
    La imagen con el contraste mejorado.
  """

  # Paso 1: Difuminar la imagen

  kernel_size = (5, 5)
  sigma = 1.0
  blurred_image = cv2.GaussianBlur(image, kernel_size, sigma)

  # Paso 2: Cierre morfológico

  kernel = np.ones((5, 5), np.uint8)
  closed_image = cv2.morphologyEx(blurred_image, cv2.MORPH_CLOSE, kernel)

  # Paso 3: División

  bg_image = closed_image.copy()
  # Evitar la división por cero
  bg_image[bg_image == 0] = 1
  divided_image = image / bg_image

  # Paso 4: Ecualización del histograma

  enhanced_image = cv2.equalizeHist(divided_image.astype(np.uint8))

  return enhanced_image



In [5]:
# Ejemplo de uso

# Variable para almacenar el directorio de la imagen
path_EL = "D:/Documentos/Universidad de Cuenca/Trabajo de Titulación/PVDefectDetect/Test_ImageJ/"
image = cv2.imread(path_EL + "example_10.png", cv2.IMREAD_GRAYSCALE)
enhanced_image = enhance_contrast(image)

cv2.imshow("Imagen original", image)
cv2.imshow("Imagen con contraste mejorado", enhanced_image)
cv2.waitKey(0)

-1